<a href="https://colab.research.google.com/github/martasannzz/hola_mundo/blob/main/class/Fundamentals/IMBD_sentiment_binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

It is highly recommended to use a powerful **GPU**, you can use it for free uploading this notebook to [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb).
<table align="center">
 <td align="center"><a target="_blank" href="https://colab.research.google.com/github/ezponda/intro_deep_learning/blob/main/class/Fundamentals/IMBD_sentiment_binary_classification.ipynb">
        <img src="https://colab.research.google.com/img/colab_favicon_256px.png"  width="50" height="50" style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/ezponda/intro_deep_learning/blob/main/class/Fundamentals/IMBD_sentiment_binary_classification.ipynb">
        <img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png"  width="50" height="50" style="padding-bottom:5px;" />View Source on GitHub</a></td>
</table>

# Classification Example
 Two-class classification, or binary classification, may be the most widely applied kind of machine-learning problem. In this example, you’ll learn to classify movie reviews as positive or negative, based on the text content of the reviews.
VAMOS A CONSTRUIR UN ANALIZADOR DE SENTIMIENTO EN TEXTOS CON UNA RED NEURONAL.
EN ESTA PRACTICA ES RECOMENDABLE USAR UNA TARJETA GRAFICA DE GOOGLE COLAB:
EDIT>NOTEBOOK SETTINGS>GPU

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
tf.random.set_seed(0)

## The Dataset: The IMDB dataset
We’ll work with the IMDB dataset: a set of 50,000 highly polarized reviews from the Internet Movie Database. They’re split into 25,000 reviews for training and 25,000 reviews for testing, each set consisting of 50% negative and 50% positive reviews. The  parameter `num_words` controls how many words different we want to use.
EL CONJUNTO QUE VAMOS A USAR ES UN CONJUNTO DE 50000 REVIEWS DE PELICULAS QUE ESTÁN MUY POLARIZADAS. LA CLASE 0 ES QUE SON NEGATIVAS Y LA CLASE 1 ES QUE SON POSITIVAS. VAMOS A TRATAR DE HACER UN CLASIFICADOR DE SENTIMIENTO PARA PREDECIR CUANDO UN TEXTO ES POSITIVO Y CUANDO UN TEXTO ES NEGATIVO.
ESTE DATASET ESTA EN KERAS Y TIENE UNOS CUANTOS DATASETS PARA HACER PRUEBAS.
TIENE UN PARÁMETRO QUE ES EL NÚMERO DE PALANRAS Y QUE ES PARA FIJAR EL NÚMERO DE PALABRAS QUE QUEREMOS PROCESAR. ESTO SIRVE PORQUE POR EJEMPLO EN ESTE DATASET QUE TIENE 60000 O 70000 PALABRAS DIFERENTES, ESO ES UN MODELO MUY GRANDE Y HABRÁ PALABRAS QUE SOLO APAREZCAN 1 SOLA VEZ. ENTONCES ASI PONEMOS EL NÚMERO MÁXIMO DE PALABRAS QUE QUEREMOS PROCESAR.


In [2]:
from tensorflow.keras.datasets import imdb
num_words = 10000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=num_words) # NOS DIVIDE EL DATASET EN ENTRENAMIENTO Y TEST.
# EN TRAIN DATA ESTÁN LAS LISTAS DE TEXTOS Y EN LABELS VA A SER 0 O 1 DEPENDIENDO DE SI ES POSITIVA O NEGATIVA.
print(train_data[0])
#UN PRIMER EJEMPLO PARA VISUALIZAR. AQUI VEMOS QUE VIENE UNNA SERIE DE INDICES PRIMERO (NO VIENEN LAS PALABRAS, YA ESTÁ PROCESADO. A CADA PALABRA/TOKEN LE ASIGA UN ÍNDICE)
# POR EJEMPLO LA PALABA HELLO PUEDE TENER EL INDICE 3, OTRA PALABRA EL 4 Y ASI SUCESIVAMENTE
# VAMOS A TENER QUE CREAR UN DICCIONARIO PARA TRANSFORMAR LOS INDICES A PALABRAS Y LAS PALABRAS A LOS INDICES. ESTO LO HACEMOS CON WORD2INT

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [ ]:
# Transform word_id to word and reverse
word2int = imdb.get_word_index()
word2int = {w: i+3 for w, i in word2int.items()} # A CADA PALABRA LE DAMOS UN INDICE Y A CADA INDICE LE DAMOS PALABRA

# HAY UNA SERIE DE PALABRAS ADICIONALES A NUESTRO VOCABULARIO
#. POR EJEMPLO EL PADING ES CADAA VEZ QUE EMPIEZA UNA LISTA Y LUEGO EL CARACTER UNK ES PARA PALABRAS QUE NO ESTAN EN NUESTRO VOCABULARIO. SI HEMOS COGIDO 10000 PALABRAS, LA 15000 NO VA A ESTAR Y ENTONCES LE ASIGAMOS EL 2 (UNKNOWN)
word2int["<PAD>"] = 0
word2int["<START>"] = 1
word2int["<UNK>"] = 2
word2int["<UNUSED>"] = 3
int2word = {i: w for w, i in word2int.items()} # LE HACEMOS LUEGO EL INVERSO PORQUE A CADA INDICE LE CORRESPONDE UNA PALABRA.
num_words = num_words+3 # LE AÑADIMOS LAS 3 PALABARS ADICIONALES A NUESTRO VOCABUALRIO

For transforming an id-sequence to a phrase use get_words

In [ ]:
# CREAMOS ESTA FUNCION QUE ES PARA TRANSFORMAR LA LISTA DE INDICES A PALABRAS. VAMOS A IR RECORRIENDO CADA INDICE DENTRO DE UNA FRASE Y LE APLICAMOS EL DICCIONARIO
def get_words(sentence, int2word):
    return ' '.join([int2word.get(i,'<UNK>') for i in sentence]) # SI NO ESTA LA PALABRA EN EL DICCIONARIO LE PONEMOS UNK.
print(get_words(train_data[0], int2word))
print('Sentiment: ', train_labels[0])

## Data Preprocessing

You need to convert your raw text to an appropriate input to a sequential model
LUEGO CREAMOS LA FUNCIÓN INVERSA PARA PASAR UN TEXTO A INDICES. POR EJEMPLO EL TEXTO ANTERIOR LO VOLVEMOS A PASAR A INDICES
ENTONCES CON ESTAS 2 FUNCIONES TENDREMOS QUE JUGAR LUEGO PARA HACER LAS PREDICCIONES.

In [ ]:
def vectorize_text_sentence(text, word2int):
    tokens = text.split(' ')
    tokens_id = [word2int.get(tk,2) for tk in tokens]
    return tokens_id

text = get_words(train_data[0], int2word)
print(text)
print(vectorize_text_sentence(text, word2int))

### Bag of Words Model BoW

HAY QUE TENER EN CUENTA QUE CADA DOCUMENTO TIENE UNA LONGITUD VARIABLE. HEMOS VISTO QUE A UNA RED NEURONAL HAY QUE DECIRLE LA DIMENSION DE ENTRADA. TENEMOS QUE USAR UNA DIMENSION FIJA. ENTONCES HABRA QUE VER UNA MANERA PARA PDER TRANSFORMAR TODOS LOS TEXTOS A UNA DIMENSION FIJA (VECTOR DE LA MISMA DIMENSION)
VAMOS A USAR EL Bag of Words Model QUE ES UNA MANERA MUY TÍPICA DE REPRESENTAR LOS DOCUMENTOS. CONSISTE EN QUE TENEMOS UN TEXTO DE ENTRADA, LUEGO LO DIVIDIMOS EN PALABRAS Y CREAMOS UNA BAG OF WORDS (BOLSA DE PALABRAS) QUE ES CADA PALABRA Y EL NÚMERO DE VECES QUE APARECE. AQUI VEMOS QUE JOHN APARECE UNA VEZ PERO LIKES APARECE 2 VECES. ESTO TAMBIEN SE PUEDE HACER NORMALIZANDO ENTRE EL NÚMERO DE PALABRAS. POR EJEMPLO AQUI TENEMOS 9 PALABRAS (1+2+1+1+2+1+1) POR LO QUE 1/9=0.11, etc.
EN ESTA MANERA DE REPRESNETAR UN DOCUMENTO, NO IMPORTA EL ORDEN. EL ORDEN SERÁ UN PROBLEMA PERO QUE GENERALMENTE VA A FUNCIONAR MUY BIEN.


We are going to use a bag of words model. BoW is a simplifying representation used in natural language processing. In this model, a text (such as a sentence or a document) is represented as the Each key is the word, and each value is the frequency of occurrences of that word in the given text document.

- **Input document**: `"John likes to watch movies Mary likes movies too"`
- **BoW**: `{'John': 1, 'likes': 2, 'to': 1, 'watch': 1, 'movies': 2, 'Mary': 1, 'too': 1}`
- **BoW Normalized**: `{'John': 0.11, 'likes': 0.22, 'to': 0.11, 'watch': 0.11, 'movies': 0.22, 'Mary': 0.11, 'too': 0.11}`


In [ ]:
from collections import Counter

# ENTONCES CON ESTA FUNCION PARTIMMOS DE UNA SECUENCIA CON COUNTER QUE NOS DA EL NÚMERO DE OCURRENCIAS DE UNA LISTA
def get_bag_of_words(sequence, norm=True):
    word_count = Counter(sequence)
    if norm: # SI LO QUEREMOS NORMALIZAR O NO.
        total = sum(word_count.values())
        word_freq = {w: n / total for w, n in word_count.items()}
        return word_freq
    else:
        return dict(word_count.items())

text_example = "John likes to watch movies Mary likes movies too"
print('text_example', text_example)
text_sequence = text_example.split()
print('text splitted', text_sequence)
bag_of_words = get_bag_of_words(text_sequence)
print('bag_of_words', bag_of_words)
print('bag_of_words norm=False', get_bag_of_words(text_sequence, norm=False))
print(
    'bag_of_words with indexes', {
        word2int[w.lower()]: p
        for w, p in get_bag_of_words(text_sequence, norm=False).items()
    })

# ESTO NOS DEVUELVE LA FRASE DEL EJEMPLO NORMALIZADA Y SIN NORMALIZAR.

### Document Term Matrix

A APRTIR DE LO ANTERIOR CREAMOS UN DOCUMENT TERM MATRIX QUE CONSISTE EN CREAR UN VECTOR DE TAMAÑO IGUAL A NUESTRO VOCABULARIO. A CADA PALABRA LE CORRESPONDERÁ UNA CELDA DE NUESTRO VECTOR.
We need a way to model the documents so that they are all the same length, so that we can use a neural network. For this we are going to use the document term matrix.
- Every document is a vector with the dimension of the vocabulary.
- The position i of the vector corresponds to the word with index i.
- The vector is all zeros except for the BoW word positions, which are filled with the frequency of the corresponding word.


POR EJEMPLO SI NEUSTRI BOCABULARIO SOLO FUERAN LAS SIGUIENTES 4 PALABRAS. DEFINIMOS QUE CADA DOCUMENTO VA A SER UN VECTOR DE 0 DE LA MISMA DIMENSION QUE EL VOCABUALRIO. LO QUE VA A TENER EN LA POSICIÓN DE CADA PALABRA ES POR EJEMPLO EL 'I' APARECE UNA VEZ  POR LO QUE LE PONEMOS 1. 'like' APARECE 1 VEZ Y LE PONEMOS 1. 'dislike' no aparece y 'movies' si por lo que ponemos 0 y 1.
PARA TEXTOS MÁS LARGOS CADA DOCUEMNTO VA A ESTAR REPRESENTADO COMO UN VECTOR DE CEROS DEL TAMAÑO DE LA DIEMNSION DEL VOCABUALRIO Y CADA PALABRA/OCURRENCIA PONDREMOS EL BNUMERO QUE APARECE. ESTO TAMBIEN SE PODRÁ NORMALIZAR


For example:

- D1 = "I like movies",  `{'I': 1, 'like': 1, 'movies': 1}`

- D2 = "I dislike movies",  `{'I': 1, 'dislike': 1, 'movies': 1}`

Then the document-term matrix would be:



   | Doc| I  | like    | dislike   | movies   |
|---:|:-------------|:-----------|:------|:------|
| D1 | 1  | 1       | 0   | 1     |
| D2 | 1  | 0    | 1   | 1     |

$D1 = [1,1,0,1]$

$D2 = [1,0,1,1]$

We convert every BoW to a vector of `dim=num_words` with `vectorize_sequences`

In [ ]:
# VAMOS A VECTORIZAR CON LAS SIGUIENTES FUNCIONES.
# DECLARAMOS UN VECTOR DE CEROS IGUAL AL TAMAÑO DE NUESTRO VOCABUALARIO, COGEMOS EL BAG OF WORDS Y VAMOS RECORRIENDO LAS PALABRAS Y LA FRECUENCIA EN LA QUE APARECEN.
# LO IREMOS PONIENDO EN LA POSICION DEL VECTOR vec[w] = freq

def vectorize_sequence(sequence, num_words, norm=True):
    vec = np.zeros(num_words)
    bow = get_bag_of_words(sequence, norm)
    for w, freq in bow.items():
        if w < num_words:
            vec[w] = freq
    return vec

# LUEGO ESTA FUNCIN CREA TODA LA MATRIZ DE TODO NUESTRO DATASET. CADA UNO DE LOS 25000 DOCUMENTOS DE TRAIN LO VAMOS A TRANSFORMAR A UN VECTOR DE DIMENSIÓN DEL TAMAÑO DEL VOCABULARIO
# EN ESTE CASO SERÁN 10000 + LOS 3 INDICES= 13000.
def vectorize_sequences(sequences, num_words=num_words, norm=True):
    """Creates an all-zero matrix of shape (len(sequences), num_words)"""
    results = np.zeros((len(sequences), num_words))
    for i, sequence in enumerate(sequences):
        results[i, :] = vectorize_sequence(sequence, num_words, norm)
    return results


x_train = vectorize_sequences(train_data, num_words=num_words)
x_test = vectorize_sequences(test_data, num_words=num_words)
y_train =np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')
x_train.shape, y_train.shape

# TRAIN SERÁ UNA MATRIZ DE 25000,10003 QUE SON LOS 25000 DOCUMENTOS POR 10003 QUE ES EL NÚMERO DE PALABRAS. Y SERÁ 25000 CEROS Y UNOS.
# A APRTIR DE AQUI YA TENEMOS UNA ENTRADA PARA LA RED NEURONAL Y A APRTIR DE AQUI VAMOS A DISEÑAR LA RED PARA PREDECIR EL SENTIMIENTO.

## Define and train a model

ENTONCES AQUI LA PRÁCTI VA A SER DISEÑARLO COMO HEMOS VISTO EN CASOS ANTERIORES O CON LA API FUNCINAL O CON LA API SECUENCIAL.
DISEÑAMOS UN MODELO QUE INTENTE CONSEGUIR 0.85 DE ACCURACY Y EN EL TEST INTENTAR LLEGAR A 0.9.
PRIMERO HAY QUE COMENZAR SOLO CON UNA CAPA, VER UNAS NEURONAS POR CAPA  E IR SUBIENDOLO HASTA IR OBTENIENDO MEJORES RESULTADOS.



Define, compile and fit your NN model

1. You can use the [Functional API](https://keras.io/guides/functional_api/):

You need to start with an input data entry:
```python    
    inputs = keras.Input(shape=(8,))
    layer_1 = layers.Dense(...)(inputs)
```

and the network outputs:
```python
outputs = layers.Dense(...)(previous_layer)
model = keras.Model(inputs=inputs, outputs=outputs)
```

2. Or you can use [Sequential API](https://keras.io/guides/sequential_model/)

```python
model = keras.Sequential(name='example_model')
model.add(layers.Dense(..., input_shape=(8,))
model.add(...
```

You can introduce regularization methods seen in [Prevent_Overfitting.ipynb](https://github.com/ezponda/intro_deep_learning/blob/main/class/Fundamentals/Prevent_Overfitting.ipynb) like [Dropout layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout):


```python
tf.keras.layers.Dropout(
    rate, noise_shape=None, seed=None, **kwargs
)
```

With Functional API:
```python
next_layer = layers.Dropout(0.4)(prev_layer)
```
With Sequential:
```python
model.add(layers.Dropout(0.4))
```

First try with only one hidden layer and see the results.

In [ ]:
model = ...

In [1]:
inputs = tf.keras.Input(shape=(num_words,), name='input_layer')  # entrada AQUI POR EJEMPLO VAMOS A PONER UNA SOLA CAPA
l_1 = layers.Dense(4,'sigmoide')(inputs) # llama a las entradas. Le ponemos por ejemplo 4 unidades y la función de activación sigmoide. Ya vimos que normalmente la mejor es la relu
# LA SALIDA COMO ES UNA CLASIFICACION BINARIA, TIENE QUE SER UNA DENSA CON UNA SIGMOIDE Y UNA SOLA NEURONA. PONEMOS LA SIGMOIDE PORQUE ES CLASIFICACION BIANRIA
outputs = layers.Dense(1,'sigmoide')(l_1)

# Model definition
model = keras.Model(inputs=inputs, outputs=outputs)

# ENTONCES YA TENDRIAMOS EL MODELO, LO COMPILAMOS:

NameError: name 'tf' is not defined

In [ ]:
es_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # can be 'val_accuracy'
    patience=5,  # if during 5 epochs there is no improvement in `val_loss`, the execution will stop
    restore_best_weights=True,
    verbose=1)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)
history = model.fit(x_train, y_train, validation_split=0.25, epochs= 25, batch_size=32, callbacks=[es_callback] # LO ENTRENAMOS SOLO CON  5 EPOCAS
                    # ES BUENA IDEA AQUI USAR CALLBACK PARA APLICAR EL EARLY STOPPING. ASI LE PODEMOS PONER MÁS EPOCAS Y QUE PARE EN EL MOMENTO QUE QUERAMOS
)

# LO COMPILAMOS CON ADAM PARA IR MONITORIZANDO LOS RESULTADOS

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
def show_loss_accuracy_evolution(hist):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Sparse Categorical Crossentropy')
    ax1.plot(hist['epoch'], hist['loss'], label='Train Error')
    ax1.plot(hist['epoch'], hist['val_loss'], label = 'Val Error')
    ax1.grid()
    ax1.legend()

    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.plot(hist['epoch'], hist['accuracy'], label='Train Accuracy')
    ax2.plot(hist['epoch'], hist['val_accuracy'], label = 'Val Accuracy')
    ax2.grid()
    ax2.legend()

    plt.show()

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
show_loss_accuracy_evolution(hist)

### Evaluate the model
You need to obtain a Test Accuracy > 0.85. Try to get more than 0.9!

VAMOS PROBANDO PORQUE EL OBJETIVO ES SUBIR DE 0.85 Y CASI LLEGAR A 0.9. ES BASTANTE SENCILLO LLEGAR CON PROBAR POCA COSA.

COMO EL MODELO ANTERIOR NO HA SIDO SUFICIENTE PODEMOS PROBAR A SUBIR EL NÚMERO DE NEURONAS POR EJEMPLO A 8 Y PODEMOS PONER UNA CAPA MÁS

In [ ]:
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

In [ ]:
def show_errors(x_test, model, labels, int2word, n_samples=10):
    preds = 1.0 * (model.predict(x_test).flatten() > 0.5)
    bad_pred_inds = np.where(preds != labels)[0]
    n_samples = min(len(bad_pred_inds), n_samples)
    samples_inds = np.random.choice(bad_pred_inds, n_samples)
    for ind in samples_inds:
        print('Predicted : {0}, real : {1}, lenght: {2}'.format(
            int(preds[ind]), labels[ind], len(test_data[ind])))
        print(get_words(test_data[ind], int2word))
        print()
    return

show_errors(x_test, model, y_test, int2word, n_samples=10)

# CON ESTA FUNCION LUEGO PODMEOS ANALIZAR LOS RESULTADOS Y VER CUANDO HA FALLADO. ES ES PORQUE HABIA POCAS PALABRAS O POR QUE

In [ ]:
# PROBAMOS ALTERNATIVAS A VER SI MEJORA

inputs = tf.keras.Input(shape=(num_words,), name='input_layer')
l_1 = layers.Dense(8,'sigmoide')(inputs)
l_2 = layers.Dense(8,'sigmoide')(l_1)
outputs = layers.Dense(1,'sigmoide')(l_2)
# ESTO YA SERÍA UN MODELO MÁS COMPLEJO. LE PODRÍAMOS PONER DROPOUT ENTRE MEDIAS:

# Model definition
model = keras.Model(inputs=inputs, outputs=outputs)

es_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    verbose=1)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'],
)
history = model.fit(x_train, y_train, validation_split=0.25, epochs= 5, batch_size=32, callbacks=[es_callback]

)

# VEMOS COMO VA MEJORANDO. PODRIAMOS SI NO PRPOBAR A PONER MÁS EPOCAS, MÁS NEURONAS...
# EL OBEJTIVO ES QUE NOSOTROS VAYAMOS PROBANDO HAYA MEJORAR UN POCO EL MODELO.

# CON LA SIGUIENTE FUNCIÓN PODEMOS VER CÓMO FUNCINARÍA EN DATOS NUEVOS
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

### Making predictioins with new data

In [ ]:
# HEMOS VISTO QUE EL MODELO BAG OF WORDS HACE QUE NO IMPORTE EL ORDEN
# AQUI VEMOS PARA CADA TEXTO EL SENTIMIENTO QUE PREDICE. YA HEMOS VIST ¡O QUE BAGS OF WORDS ESTO NO LO VA A PREDECIR BIEN PORQUE NO DETECTA EL ORDEN
# POR EJEMPLO LA FRASE the movie is not bad I like it ES IGUAL QUE SI PONDRIA the movie is bad I dont like it


reviews = ['the film was really bad and i am very disappointed',
           'The film was very funny entertaining and good we had a great time . brilliant film',
           'this film was just brilliant',
           'the movie is not bad',
           'the movie is not bad I like it'
]
sequences = [vectorize_text_sentence(review.lower(), word2int)
             for review in reviews]

x_pred = vectorize_sequences(sequences, num_words=num_words)
predictions = model.predict(x_pred)
for review, pred in zip(reviews, predictions.flatten()):
    print()
    print(review)
    print('Sentiment: ', np.round(pred, 2))

In [ ]:
1.0*(model.predict(x_pred) > 0.5)

### Question 1: Repeat the process with unnormalized bag of words, compare results
PARA VER LA FEATURE ENGINEERING VAMOS A VER QUE PASA SI NO NORMALIZAMOS LOS BAG OF WORDS Y LO DEJAMOS COMO EL CONTEO QUÉ OCURRE.
PODEMOS TENER UN TEXTO EN EL QUE LA PALABRA 'buena' APAREZCA UNA VEZ PERO SEA MUY CORTO. SIN EMBARGO TAMBIEN PUEDE HABER UN TEXTO QUE TENGA 10000 PALABRAS Y QUE LA PALABRA BUENA SOLO APAREZCA UNA VEZ.
SI NORMALIZAMOS ESTAREMOS PONIENDOLO EN RELACION CON LA LONGITUD DEL DOCUMENTO. SI NO DARÍA IGUAL. LOS DOS TENDRÍAN EL MISMO VALOR. ENTONCES CON EL MODELO ANTERIOR, QUE HAYQUE VOLVER A EJECUATRLO CADA VEZ QUE LO QUERAMOS VOLVER A ENTRENAR PORQUE SI NO SE QUEDA CON LOS PESOS ANTERIORES.
SI LO ENTRENAMOS, VEREMOS QUE A VECES TIENE PEOR RESULATDO SIN NORMALIZAR.

In [ ]:
x_train = vectorize_sequences(train_data, num_words=num_words, norm=False)
x_test = vectorize_sequences(test_data, num_words=num_words, norm=False)
y_train =np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')
x_train.shape, y_train.shape

In [ ]:
model = ...

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history = model.fit(x_train, y_train, validation_split=0.25, epochs= 10, batch_size=32)
show_loss_accuracy_evolution(hist)
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

### Question 2: What happens if you reduce the size of the vocabulary `num_words`, compare results
COMO PRÁCTICA NOS HA DEJADO REDUCIR EL NÚMERO DE VOCABUALRIO, AUMENTARLO Y COMPARAR LOS RESULTADOS PARA VER SI EL PROBLEMA ERA DE ESO.
LUEGO COMO CURIOSIDAD PODEMOS COMPARARLO CON OTROS MODELOS DE ML Y VER QUE SI HEMOS HECHO UNA RED LO SUFICIENTEMENTE BUENA LOS VA MEJORANDO GENERALMENTE.

In [ ]:
num_words = ...
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=num_words)
print(train_data[0])

word2int = imdb.get_word_index()
word2int = {w: i+3 for w, i in word2int.items()}
word2int["<PAD>"] = 0
word2int["<START>"] = 1
word2int["<UNK>"] = 2
word2int["<UNUSED>"] = 3
int2word = {i: w for w, i in word2int.items()}
num_words = num_words+3

x_train = vectorize_sequences(train_data, num_words=num_words)
x_test = vectorize_sequences(test_data, num_words=num_words)
y_train =np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')
x_train.shape, y_train.shape

In [ ]:
model = ...

In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
history = model.fit(x_train, y_train, validation_split=0.25, epochs= 10, batch_size=32)
show_loss_accuracy_evolution(hist)
results = model.evaluate(x_test, y_test, verbose=1)
print('Test Loss: {}'.format(results[0]))
print('Test Accuracy: {}'.format(results[1]))

In [ ]:
reviews = [
    'the film was really bad and i am very disappointed',
    'The film was very funny entertaining and good we had a great time . brilliant film',
    'this film was just brilliant', 'the movie is not bad',
    'the movie is not bad I like it'
]
sequences = [
    vectorize_text_sentence(review.lower(), word2int) for review in reviews
]

x_pred = vectorize_sequences(sequences, num_words=num_words)
predictions = model.predict(x_pred)
for review, pred in zip(reviews, predictions.flatten()):
    print()
    print(review)
    print('Sentiment: ', np.round(pred, 2))

### Compare with other ML algorithms
LUEGO COMO CURIOSIDAD PODEMOS COMPARARLO CON OTROS MODELOS DE ML Y VER QUE SI HEMOS HECHO UNA RED LO SUFICIENTEMENTE BUENA LOS VA MEJORANDO GENERALMENTE.

In [ ]:
from sklearn.linear_model import LogisticRegression
clr = LogisticRegression()
clr.fit(x_train, y_train)
val_acc = clr.score(x_test, y_test)
print('Test Accuracy: {}'.format(val_acc))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=5, n_jobs=-1)
rfc.fit(x_train, y_train)
val_acc = rfc.score(x_test, y_test)
print('Test Accuracy: {}'.format(val_acc))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_jobs=-1)
rfc.fit(x_train, y_train)
val_acc = rfc.score(x_test, y_test)
print('Test Accuracy: {}'.format(val_acc))

In [ ]:
reviews = []
# training reviews
for ind in range(len(train_data)):
    sentence = train_data[ind]
    sentence_text = get_words(train_data[ind], int2word)
    reviews.append(sentence_text)
print('First training review: ', reviews[0])